In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../Datasets/ETH.csv")

In [3]:
df.head()

,Crypto,Date,Open,High,Low,Close
0,ETH,12/31/21,3736.59,3812.67,3622.51,3687.67
1,ETH,12/30/21,3725.60,3768.63,3590.00,3733.98
2,ETH,12/29/21,3809.14,3827.01,3700.00,3724.98
3,ETH,12/28/21,4061.29,4075.97,3765.23,3807.82
4,ETH,12/27/21,4086.36,4128.28,4037.00,4061.30


In [4]:
print("Dataframe Shape: " , df.shape)
print("Dataframe feature Datatypes: ", df.dtypes)
print("Dataframe Description: ", df.describe())

Dataframe Shape:  (1975, 6)
Dataframe feature Datatypes:  Crypto     object
Date       object
Open      float64
High      float64
Low       float64
Close     float64
dtype: object
Dataframe Description:                Open         High          Low        Close
count  1975.000000  1975.000000  1975.000000  1975.000000
mean   1187.961544  1226.940577  1144.599033  1188.435453
std    1164.574631  1200.438606  1123.993771  1164.606959
min      81.720000    84.150000    80.920000    81.720000
25%     215.730000   222.820000   207.525000   215.615000
50%     646.980000   669.000000   618.120000   647.000000
75%    1817.655000  1855.245000  1773.655000  1817.660000
max    4800.000000  4865.810000  4715.510000  4800.000000


In [5]:
df.isnull().sum()

Crypto    0
Date      0
Open      0
High      0
Low       0
Close     0
dtype: int64